In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from numpy import count_nonzero

dir_ = '../data/'
# file_name = 'normalized_filter_track_5_user_100.csv'
# file_name = 'normalized_to_rating_filter_track_20_user_100.csv'
file_name = 'normalized_popularity_filter_track_5_user_100.csv'

In [2]:
prediction_count = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_count_top_N_' + file_name[:-3] + 'pkl'))
prediction_rating = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_rating_top_N_' + file_name[:-3] + 'pkl'))

In [3]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
pd_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_normalized_to_rating_filter_track_5_user_100.pkl'

In [4]:
pd_train

NameError: name 'pd_train' is not defined

In [5]:
prediction_count

,tid,count,rating
166,166,1882,333.50
6310,6338,1589,92.25
17316,17377,1555,43.50
6254,6282,1396,17.50
13055,13104,1340,33.50
...,...,...,...
41773,42054,1,0.25
53483,53940,1,0.25
35279,35475,1,0.25
15826,15881,1,0.25


In [6]:
prediction_rating

,tid,count,rating
166,166,1882,333.50
348,350,1291,272.00
3748,3761,976,258.00
7178,7209,805,256.50
80,80,1207,217.50
...,...,...,...
49206,49598,1,0.25
49194,49586,1,0.25
8874,8909,1,0.25
8870,8905,1,0.25


In [7]:
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])
test

,uid,tid,rating
481832,90,5374,0.25
1247457,214,35528,1.25
968433,172,29367,4.25
25524,4,11632,4.25
60178,8,19012,0.25
...,...,...,...
587045,115,27784,3.25
309534,65,12495,0.25
660480,130,20821,0.25
373255,73,27444,2.75


In [8]:
test

,uid,tid,rating
481832,90,5374,0.25
1247457,214,35528,1.25
968433,172,29367,4.25
25524,4,11632,4.25
60178,8,19012,0.25
...,...,...,...
587045,115,27784,3.25
309534,65,12495,0.25
660480,130,20821,0.25
373255,73,27444,2.75


In [9]:
num_user = 211
num_user

211

In [10]:
#############################################
#                                           #
#          Personal Recommendation          #
#                                           #
#############################################

In [11]:
n=4

top_n_list_r = []
top_n_list_c = []

prediction_r = prediction_rating[:n]
prediction_c = prediction_count[:n]

for _, row in prediction_r.iterrows():
    top_n_list_r.append(row[0])
    
for _, row in prediction_c.iterrows():
    top_n_list_c.append(row[0])

print(top_n_list_c)
print(top_n_list_r)

[166.0, 6338.0, 17377.0, 6282.0]
[166.0, 350.0, 3761.0, 7209.0]


In [12]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0


for i in tqdm.tqdm_notebook(range(num_user)):
    precision_r = 0
    precision_c = 0
    for j in top_n_list_r:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision_r += 1
            
    for j in top_n_list_c:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision_c += 1    
            
    satisfication_r += precision_r / n
    satisfication_c += precision_c / n

    
    evaluation_r.append(precision_r)
    evaluation_c.append(precision_c)

#     print(precision, satisfication)
print('rating', satisfication_r/num_user)
print('count',satisfication_c/num_user)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



rating 0.31398104265402843
count 0.16587677725118483


In [37]:
#############################################
#                                           #
#          Group Recommendation             #
#                                           #
#############################################

In [13]:
groups = []
group_size = 2

for i in range(num_user//group_size):
    groups.append([])
    
for i in range(num_user):
    b = True
    while b:
        g = random.random()*(num_user/group_size)//1
        group = groups[int(g)]
        if len(group) < group_size:
            group.append(i)
            b = False
groups

IndexError: list index out of range

In [102]:
# check if duplicate 

users = []
for group in groups:
    for u in group:
        users.append(u)
users.sort(reverse=True)

n = num_user - 1 
for i in users:
    if i-n != 0:
        print('False')
    n-=1

In [ ]:
#############################################
#                                           #
#          Group Recommendation (2)         #
#                                           #
#############################################

In [23]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_2.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([32., 78.])

In [27]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            precision_r += 1
#         precision_r += math.log(s+1,10)/math.log(len(group)+1,10)

    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision_c += 1
#             precision_c += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


0.25
0.0
0.25
0.25
0.5
0.0
0.25
0.0
0.25
0.25
0.5
0.25
0.75
0.25
0.25
0.25
0.5
0.0
0.5
0.0
0.5
0.25
0.25
0.75
0.5
0.0
0.5
0.0
0.25
0.25
0.25
0.5
0.5
0.75
0.0
0.0
0.25
0.25
0.5
0.0
0.25
0.25
0.25
0.5
0.0
0.25
0.5
0.5
0.25
0.0
0.5
0.5
0.0
0.0
0.25
0.5
0.5
0.5
0.0
0.0
0.25
0.5
0.25
0.0
0.5
0.25
0.25
0.5
0.0
0.0
0.0
0.5
0.25
0.5
0.5
0.0
0.25
0.25
0.75
0.25
0.25
0.5
0.5
0.0
0.0
0.5
0.0
0.25
0.5
0.0
0.5
0.5
0.25
0.0
0.0
0.5
0.75
0.5
0.25
0.0
0.0
0.25
0.25
0.25
0.25

rating 0.5190476190476191
count 0.28095238095238095


In [24]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
    
print('count', satisfication/(num_user//group_size))


count 0.1326705202789367


In [ ]:
#############################################
#                                           #
#          Group Recommendation (4)         #
#                                           #
#############################################

In [20]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_4.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 16.,  69., 111., 129.])

In [22]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
#         if s > 0:
#             precision_r += 1
        precision_r += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision_c += 1
#             precision_c += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



rating 0.44526492397020273
count 0.4326923076923077


In [32]:
# modified satisfaction
satisfication = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0    
    high_rating_u3 = 0    
    high_rating_u4 = 0    

    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u4 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)    
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)   

    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
    
print('count', satisfication/(num_user//group_size))


count 0.14332027127647462


In [145]:
#############################################
#                                           #
#          Group Recommendation (5)         #
#                                           #
#############################################

In [34]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_5_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 11.,  23., 107., 122., 141.])

In [35]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
#         if s > 0:
#             precision_r += 1
        precision_r += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision_c += 1
#             precision_c += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))


rating 0.47385832035729325
count 0.30357142857142855


In [41]:
# modified satisfaction
satisfication = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0    
    high_rating_u3 = 0    
    high_rating_u4 = 0    
    high_rating_u5 = 0    

    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]

        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u4 += 1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u5 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)    
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)   
    satisfication_u += math.log(high_rating_u5+1,10)/math.log(n+1,10)   

    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
    
print('count', satisfication/(num_user//group_size))


count 0.26164266772875894


In [111]:
#############################################
#                                           #
#          Group Recommendation (10)        #
#                                           #
#############################################

In [42]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_10_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 17.,  58.,  73., 104., 112., 164., 207., 214., 218., 219.])

In [177]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            precision_r += 1
#         precision_r += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t6) > 0 and t6.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t7) > 0 and t7.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t8) > 0 and t8.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t9) > 0 and t9.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t10) > 0 and t10.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision_c += 1
#             precision_c += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))


rating 0.9523809523809523
count 0.42857142857142855


In [47]:
# modified satisfaction
satisfication = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0    
    high_rating_u3 = 0    
    high_rating_u4 = 0    
    high_rating_u5 = 0    
    high_rating_u6 = 0
    high_rating_u7 = 0    
    high_rating_u8 = 0    
    high_rating_u9 = 0    
    high_rating_u10 = 0    
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]

        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u4 += 1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u5 += 1
            if len(t6) > 0 and t6.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u6 += 1
            if len(t7) > 0 and t7.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u7 += 1
            if len(t8) > 0 and t8.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u8 += 1
            if len(t9) > 0 and t9.iloc[0]['rating'] > rating:
                high_rating += 1
                high_rating_u9 += 1
            if len(t10) > 0 and t10.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u10 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)    
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)   
    satisfication_u += math.log(high_rating_u5+1,10)/math.log(n+1,10)   
    satisfication_u += math.log(high_rating_u6+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u7+1,10)/math.log(n+1,10)    
    satisfication_u += math.log(high_rating_u8+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u9+1,10)/math.log(n+1,10)   
    satisfication_u += math.log(high_rating_u10+1,10)/math.log(n+1,10)   

    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
    
print('count', satisfication/(num_user//group_size))


count 0.16489336140758612


In [185]:
#############################################
#                                           #
#          Distribution of rating           #
#                                           #
#############################################

In [317]:
dir_ = '../data_0.5_track_20/'
# file_name = 'normalized_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_20_user_100.csv'

In [178]:
ratings = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
rating_list = []

for i in range(1,6):
    rating = ratings[ratings['rating']<=i] 
    num = len(rating)
    rating_list.append(num)
    print(num)
    
print('')

for i in range(4,0,-1):
    rating_list[i] -= rating_list[i-1]
    
for i in range(5):
    print(rating_list[i])

294240
368197
472624
583652
702362

294240
73957
104427
111028
118710


In [179]:
ratings

,uid,tid,rating
4,0,0,4.75
5,0,1,2.50
6,0,2,0.25
7,0,3,0.25
8,0,4,4.50
...,...,...,...
1256928,219,54824,4.75
1256936,219,54867,1.75
1256949,219,54900,0.25
1256989,219,54958,0.25
